In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
def make_soup(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

In [3]:
class pbp_drive():
    
    def parse_drive(self,drive):
        
        # Try to read drive header
        header = drive.find("div",{"class":"accordion-header"})
        self.is_half = False
        if header == None: # Then we've got the end of a half or something
            self.is_half = True
#            print("found end of half, etc.")
            text = drive.find("span",{"class":"post-play"}).contents
            df = pd.DataFrame([[[],text]], columns=['downdist','detail'])
            return df
        
        # Grab information from the drive header
        possessor_logo = drive.find("span",{"class":"home-logo"}).contents[0]
        s = "nfl/500/"
        e = ".png"
        # Cut off pieces of url before and after home team
        self.offense = (str(possessor_logo).split(s))[1].split(e)[0].upper()
        # Get result of the drive
        self.result = header.find("span",{"class":"headline"}).contents
        # Get info about home/away score
        home_info = header.find("span",{"class":"home"}).contents
        self.home_team = home_info[0].contents[0]
        self.home_score_after = home_info[1].contents[0]
        away_info = header.find("span",{"class":"away"}).contents
        self.away_team = away_info[0].contents[0]
        self.away_score_after = away_info[1].contents[0]
        # Get drive summary
        self.drive_detail = header.find("span",{"class":"drive-details"}).contents
#        print(self.offense)
#        print(self.drive_detail)
#        print(self.result)
#        print([self.home_team,self.home_score_after,self.away_team,self.away_score_after])
    
        # Make a dataframe for the drive
        # Grab info about individual plays from this drive
        playlist = []
        plays = drive.find_all("li")
        for p in plays:
            try:
                downdist = p.h3.contents
                detail = p.span.contents[0].replace("\n","").replace("\t","")
                playlist.append([downdist,detail])
#                print([downdist,detail])
            except:
                pass
            
        df = pd.DataFrame(playlist, columns=['downdist','detail'])
        return df


In [4]:
# Putting all of the pieces together
def get_game_df(url):
    
    # Make a soup object with html
    soup = make_soup(url)
    
    # Find article with play-by-play table
    article = soup.find("article", {"class":"sub-module play-by-play"})
    # Article is constructed like accordion, with items corresponding
    # to individual drives
    accordion = article.find("ul", {"class":"css-accordion"})
    drives = accordion.find_all("li", {"class":"accordion-item"})
    
    # Now parse each of the drives into a dataFrame
    drivelist = []
    for i, drive in enumerate(drives):
        # Initialize drive object, then parse 
        d = pbp_drive()
        d.df = d.parse_drive(drive)
    
        if i == 0:
            d.home_score = 0
            d.away_score = 0
        else:
            d.home_score = drivelist[-1].home_score_after
            d.away_score = drivelist[-1].away_score_after
    
        # If the drive isn't a special section marking the end of half/game
        # Then add drive's dataFrame to the drive list
        if not d.is_half:
            d.df['home'] = d.home_team
            d.df['away'] = d.away_team
            d.df['possession'] = d.offense
            d.df['home_score'] = d.home_score
            d.df['away_score'] = d.away_score

            #print(d.df)
            drivelist.append(d)
            
    game_df = pd.concat([d.df for d in drivelist])
    return game_df

In [5]:
game = get_game_df("http://www.espn.com/nfl/playbyplay?gameId=400951568")
game.head(15)

,downdist,detail,home,away,possession,home_score,away_score
0,[],(15:00 - 1st) G.Zuerlein kicks 65 yards from L...,LAR,SF,SF,0,0
1,[1st and 10 at SF 25],(14:52 - 1st) B.Hoyer pass short right intend...,LAR,SF,SF,0,0
0,[1st and Goal at SF 3],(14:48 - 1st) Todd Gurley 3 Yard Rush G.Zuerle...,LAR,SF,LAR,0,0
0,[],(14:43 - 1st) G.Zuerlein kicks 64 yards from L...,LAR,SF,SF,7,0
1,[1st and 10 at SF 19],(14:31 - 1st) C.Hyde up the middle to SF 21 f...,LAR,SF,SF,7,0
2,[2nd and 8 at SF 21],(14:01 - 1st) C.Hyde left tackle to SF 41 for...,LAR,SF,SF,7,0
3,[1st and 10 at SF 41],(13:29 - 1st) C.Hyde right guard to SF 44 for...,LAR,SF,SF,7,0
4,[2nd and 7 at SF 44],(13:14 - 1st) M.Breida left tackle to SF 41 f...,LAR,SF,SF,7,0
5,[3rd and 10 at SF 41],(12:33 - 1st) (Shotgun) B.Hoyer pass short ri...,LAR,SF,SF,7,0
6,[4th and 2 at SF 49],(11:57 - 1st) B.Pinion punts 36 yards to LA 1...,LAR,SF,SF,7,0


In [7]:
# Function to get gameIds for a particular year/week
results = {}
def get_gameId(year,week):
    # Make a soup object for the appropriate page
    url = "http://www.espn.com/nfl/schedule/_/week/{0}/year/{1}".format(week,year)
    soup = make_soup(url)
    sched_page = soup.find("section",{"id":"main-container"})
    
    # Make a list for gameIds
    gameids = []
    for link in sched_page.find_all('a'):
        if "gameId" in link.get('href'):
            # Extract last bit of url listed
            s = "gameId="
            this_game = link.get('href').split(s)[1]
            gameids.append(this_game)
            # And add text displayed to a dictionary
            results[this_game] = link.contents[0]
    
    return gameids, results

In [8]:
gameids, results = get_gameId(2017,1)
print(gameids)
print(results)

['400951566', '400951567', '400951570', '400951572', '400951574', '400951576', '400951584', '400951592', '400951580', '400951597', '400951601', '400951605', '400951608', '400951581', '400951612', '400951615']
{'400951566': 'KC 42, NE 27', '400951567': 'BUF 21, NYJ 12', '400951570': 'ATL 23, CHI 17', '400951572': 'BAL 20, CIN 0', '400951574': 'PIT 21, CLE 18', '400951576': 'DET 35, ARI 23', '400951584': 'OAK 26, TEN 16', '400951592': 'PHI 30, WSH 17', '400951580': 'JAX 29, HOU 7', '400951597': 'LAR 46, IND 9', '400951601': 'GB 17, SEA 9', '400951605': 'CAR 23, SF 3', '400951608': 'DAL 19, NYG 3', '400951581': 'Postponed', '400951612': 'MIN 29, NO 19', '400951615': 'DEN 24, LAC 21'}


In [9]:
# Loop over desired weeks/years to get gameIds that can be used to look up play-by-play for each of the games
import time
gameids = []
gameresults = {}
gameyear = {}
gameweek = {}
for year in range(2017,2018):
    for week in range(6,11):
        print("Looking up gameIds for {0} week {1}".format(year,week))
        ids, results = get_gameId(year,week)
        gameids.append(ids)
        
        # Add entry in dictionaries for each gameId
        for i in ids:
            gameyear[i] = year
            gameweek[i] = week
            gameresults[i] = results[i]
        
        # Sleep so we don't get blocked
        time.sleep(0.5)
        
# Now flatten gameids, which is a list of weekly lists
ids = [i for sublist in gameids for i in sublist]
print(gameresults)

Looking up gameIds for 2017 week 6
Looking up gameIds for 2017 week 7
Looking up gameIds for 2017 week 8
Looking up gameIds for 2017 week 9
Looking up gameIds for 2017 week 10
{'400951695': 'PHI 28, CAR 23', '400951697': 'MIA 20, ATL 17', '400951702': 'MIN 23, GB 10', '400951704': 'NO 52, DET 38', '400951766': 'NE 24, NYJ 17', '400951767': 'WSH 26, SF 24', '400951699': 'CHI 27, BAL 24 (OT)', '400951700': 'HOU 33, CLE 17', '400951770': 'ARI 38, TB 33', '400951773': 'LAR 27, JAX 17', '400951776': 'PIT 19, KC 13', '400951779': 'LAC 17, OAK 16', '400951782': 'NYG 23, DEN 10', '400951785': 'TEN 36, IND 22', '400951571': 'OAK 31, KC 30', '400951575': 'BUF 30, TB 27', '400951578': 'CHI 17, CAR 3', '400951582': 'TEN 12, CLE 9 (OT)', '400951585': 'NO 26, GB 17', '400951589': 'JAX 27, IND 0', '400951593': 'LAR 33, ARI 0', '400951598': 'MIA 31, NYJ 28', '400951603': 'MIN 24, BAL 16', '400951619': 'DAL 40, SF 10', '400951628': 'SEA 24, NYG 7', '400951609': 'PIT 29, CIN 14', '400951624': 'LAC 21, D

In [10]:
# Make dataframe for game-specific information using our dictionaries for year and week
data = [ {'gameId':i, 'season':gameyear[i], 'week':gameweek[i], 'result':gameresults[i]} for i in ids]
gamedata_df = pd.DataFrame(data)
# Set the gameId as the unique identifier for each row
gamedata_df.set_index('gameId', inplace=True)
gamedata_df.sample(10)

,result,season,week
gameId,,,
400951782,"NYG 23, DEN 10",2017,6
400951670,"BAL 40, MIA 0",2017,8
400951715,"NE 21, LAC 13",2017,8
400951693,"CAR 45, MIA 21",2017,10
400951582,"TEN 12, CLE 9 (OT)",2017,7
400951690,"NE 41, DEN 16",2017,10
400951773,"LAR 27, JAX 17",2017,6
400951641,"PHI 34, WSH 24",2017,7
400951776,"PIT 19, KC 13",2017,6


In [11]:
# Create list of individual game dataFrames
dflist = []
game_home = {}
game_away = {}

# Now loop over gameIds to scrape individual game play-by-play
for i in ids:
    
    # Check year of game. Only search for pbp of games from 2004 or later.
    if gameyear[i] >= 2004:
        
        try:
    
            print(i)
            # Make whole url for play-by-play
            url = "http://www.espn.com/nfl/playbyplay?gameId="+i
    
            # Call function to scrape and parse info into a dataFrame
            pbp_df = get_game_df(url)
    
            # Add column to dataframe for gameId
            pbp_df['gameId'] = i
    
            # Extract home/away from game df
            game_home[i] = pbp_df['home'].values[0]
            game_away[i] = pbp_df['away'].values[0]

            dflist.append(pbp_df)
            
        except:
            print("Failed to scrape gameId "+i)
            pass

    
# Put individual game dataFrames together into one big dataFrame
allplays_df = pd.concat(dflist)

400951695
400951697
400951702
400951704
400951766
400951767
400951699
400951700
400951770
400951773
400951776
400951779
400951782
400951785
400951571
400951575
400951578
400951582
400951585
400951589
400951593
400951598
400951603
400951619
400951628
400951609
400951624
400951638
400951641
400951670
400951683
400951706
400951711
400951715
400951717
400951721
400951723
400951725
400951729
400951732
400951736
400951737
400951743
400951761
400951755
400951758
400951760
400951749
400951753
400951751
400951763
400951765
400951786
400951787
400951790
400951553
400951555
400951559
400951563
400951656
400951565
400951653
400951658
400951648
400951663
400951686
400951688
400951690
400951693


In [12]:
# Add columns to gamedata_df for home and away team
gamedata_df['home'] = pd.Series(game_home)
gamedata_df['away'] = pd.Series(game_away)

print(gamedata_df[gamedata_df['season']>=2004].head(5))
allplays_df.sample(10)

                   result  season  week home away
gameId                                           
400951695  PHI 28, CAR 23    2017     6  PHI  CAR
400951697  MIA 20, ATL 17    2017     6  MIA  ATL
400951702   MIN 23, GB 10    2017     6   GB  MIN
400951704   NO 52, DET 38    2017     6  DET   NO
400951766   NE 24, NYJ 17    2017     6   NE  NYJ


,downdist,detail,home,away,possession,home_score,away_score,gameId
1,[1st and 10 at NYG 25],(2:14 - 4th) (Shotgun) E.Manning pass incompl...,SEA,NYG,NYG,24,7,400951628
0,[1st and 10 at KC 13],(8:37 - 1st) K.Hunt right tackle to KC 13 for...,KC,DAL,KC,0,0,400951786
3,[2nd and 16 at PHI 19],(13:59 - 3rd) (Shotgun) C.Wentz pass short le...,SF,PHI,PHI,0,17,400951723
1,[2nd and 4 at PHI 26],(6:20 - 2nd) (Shotgun) W.Smallwood up the mid...,WSH,PHI,PHI,10,3,400951641
13,[4th and Goal at GB 1],(12:31 - 4th) Matt Prater 19 Yd Field Goal,DET,GB,DET,17,3,400951790
8,[2nd and 8 at DAL 11],(1:37 - 2nd) M.Ryan pass incomplete short rig...,DAL,ATL,ATL,7,3,400951686
4,[2nd and 4 at JAX 44],(9:28 - 3rd) T.Yeldon up the middle to JAX 49...,CIN,JAX,JAX,7,13,400951753
0,[1st and 10 at GB 8],(11:57 - 1st) (Shotgun) A.Jones left tackle t...,GB,CHI,GB,0,0,400951559
2,[3rd and 4 at CAR 13],(8:35 - 3rd) (Shotgun) C.Newton pass incomple...,CAR,TB,CAR,10,0,400951725
2,[2nd and 10 at DEN 25],(8:40 - 2nd) (Shotgun) B.Osweiler pass short ...,DEN,PHI,DEN,6,24,400951760


## Begin processing the dataFrames

In [13]:
# Get information about winning team and final scores
winner = {}
home_score = {}
away_score = {}
ot = {}
for i in list(gamedata_df.index.values):
    try:
        final = gameresults[i]
    
        # Winner should be first team listed
        winner[i] = final.split()[0]
    
        if "(OT)" in final:
            ot[i] = 1
        else:
            ot[i] = 0
        
        if game_home[i] == winner[i]:
            # Home team wins, their score is listed first
            home_score[i] = final.split()[1].rstrip(",")
            away_score[i] = final.split()[3]
        else:
            # Away team wins, their score is listed first
            home_score[i] = final.split()[3]
            away_score[i] = final.split()[1].rstrip(",")
        
        # Check for a tie
        if home_score[i] == away_score[i]:
            winner[i] = "TIE"
            
    except:
        winner[i] = "unknown"
        ot[i] = "unknown"
        home_score[i] = "unknown"
        away_score[i] = "unknown"
    
gamedata_df['winner'] = pd.Series(winner)
gamedata_df['home_score'] = pd.Series(home_score)
gamedata_df['away_score'] = pd.Series(away_score)
gamedata_df['OT'] = pd.Series(ot)

In [14]:
gamedata_df.sample(15)

,result,season,week,home,away,winner,home_score,away_score,OT
gameId,,,,,,,,,
400951736,"PIT 20, DET 15",2017,8,PIT,DET,PIT,20,15,0
400951706,"BUF 34, OAK 14",2017,8,OAK,BUF,BUF,14,34,0
400951700,"HOU 33, CLE 17",2017,6,CLE,HOU,HOU,17,33,0
400951555,"NO 47, BUF 10",2017,10,NO,BUF,NO,47,10,0
400951782,"NYG 23, DEN 10",2017,6,NYG,DEN,NYG,23,10,0
400951575,"BUF 30, TB 27",2017,7,TB,BUF,BUF,27,30,0
400951589,"JAX 27, IND 0",2017,7,JAX,IND,JAX,27,0,0
400951786,"DAL 28, KC 17",2017,9,KC,DAL,DAL,17,28,0
400951683,"MIN 33, CLE 16",2017,8,MIN,CLE,MIN,33,16,0


## And start working with the play-by-play data

In [15]:
# Start by saving both dataFrames to disk in case I mess anything up
gamedata_df.to_csv("espn_2017gamedata_wk6-10.csv")
allplays_df.to_csv("espn_rawplays_2017_wk6-10.csv")

In [16]:
# Load dataframes from disk
gamedata_df = pd.read_csv("espn_2017gamedata_wk6-10.csv")
allplays_df = pd.read_csv("espn_rawplays_2017_wk6-10.csv")

In [17]:
# Take another look at what we've got so far
gamedata_df.set_index('gameId', inplace=True)
print(allplays_df.info())
allplays_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 9 columns):
Unnamed: 0    11935 non-null int64
downdist      11935 non-null object
detail        11935 non-null object
home          11935 non-null object
away          11935 non-null object
possession    11935 non-null object
home_score    11935 non-null int64
away_score    11935 non-null int64
gameId        11935 non-null int64
dtypes: int64(4), object(5)
memory usage: 839.3+ KB
None


,Unnamed: 0,downdist,detail,home,away,possession,home_score,away_score,gameId
6074,5,['1st and 10 at NYJ 48'],(12:39 - 1st) (Shotgun) B.Powell up the middl...,ATL,NYJ,NYJ,0,0,400951721
7363,0,['1st and 10 at BUF 25'],(3:18 - 2nd) R.Groy reported in as eligible. ...,BUF,NYJ,BUF,7,10,400951743
765,3,['3rd and 8 at NE 27'],(11:28 - 2nd) (Shotgun) T.Brady pass incomple...,NE,NYJ,NE,0,14,400951766
4780,1,['1st and 10 at ATL 28'],(14:54 - 3rd) T.Coleman left guard to ATL 30 ...,ATL,NE,ATL,0,17,400951638
11636,0,[],(5:18 - 1st) B.McManus kicks 65 yards from DEN...,NE,DEN,NE,14,6,400951690


In [18]:
# Start by trying to parse down, distance, and field position

# Make lists to populate with a value for each play
down = []
dist = []
home_fieldpos = []

# Make lists out of home and away teams for comparison with field position
hometeam = allplays_df.home.values
awayteam = allplays_df.away.values

# Function to return fieldposition from the offense's point of view
def get_fieldpos(teamside,ydline,j):
    if teamside == hometeam[j]:
        # Ball is one home team's half. Location should be negative
        return -1*(50-ydline)
    elif teamside == awayteam[j]:
        return 50-ydline
    else:
        return "x"
        

for j, c in enumerate(allplays_df['downdist'].values):
    
    x = c.strip("[]'")
    # Check for an empty list. This probably means an end of quarter/half line
    if (not x) or x == None:
        down.append(0)
        dist.append(0)
        home_fieldpos.append(0)
#        print("Found empty list")
        
    else:
        
        x = [x]
        pieces = x[0].split()
#        print(pieces)
    
        # Get down
        if not pieces[0][0].isalpha():  # check is first character is alphabetic
            # Then first character is numeric. This is the down number.
            down.append(int(pieces[0][0]))
        else:
            down.append(0)
        
        # Get distance
        for i, word in enumerate(pieces):
            if word == "and":
                dist.append(pieces[i+1])  # Keep as string to preserve goal-to-go situations
        
        # Get fieldposition from the home team's perspective
        for i, word in enumerate(pieces):
            if word == "at":
                if pieces[i+1] == '50':
                    home_fieldpos.append(0)
                else:
                    teamside = pieces[i+1]
                    ydline = int(pieces[i+2])
                    
                    fieldpos = get_fieldpos(teamside,ydline,j)
                    
                    if fieldpos == "x":  # Failed to match teamside with home/away teams
                        # Change teamside in a couple cases to account for teams moving
                        # ESPN seems to always use most recent short form name in fieldposition
                        if teamside == "LAR":
                            teamside = "STL"
                        elif teamside == "LAC":
                            teamside = "SD"
                        # Try again with new teamside
                        fieldpos = get_fieldpos(teamside,ydline,j)
                    
                    if fieldpos == "x":
                        home_fieldpos.append(0)
                        print(pieces)
                        print("Failed to find side of field correctly")
                        
                    else:
                        home_fieldpos.append(fieldpos)
                        
                
#    print([down[-1], dist[-1], home_fieldpos[-1]])
allplays_df['down'] = down
allplays_df['dist'] = dist
allplays_df['home_fieldpos'] = home_fieldpos

In [19]:
allplays_df.sample(5)

,Unnamed: 0,downdist,detail,home,away,possession,home_score,away_score,gameId,down,dist,home_fieldpos
11896,5,['1st and 10 at 50'],(0:57 - 3rd) (Shotgun) J.Cutler pass incomple...,MIA,CAR,MIA,14,38,400951693,1,10,0
6385,3,['2nd and 10 at PHI 49'],(12:34 - 4th) (Shotgun) C.Wentz pass short mi...,SF,PHI,PHI,7,27,400951723,2,10,1
11540,0,['1st and 10 at SF 24'],(9:39 - 3rd) C.Hyde right tackle to SF 27 for...,NYG,SF,SF,13,17,400951688,1,10,26
3972,2,['1st and 10 at BAL 22'],(13:28 - 4th) J.McKinnon right end to BLT 18 ...,BAL,MIN,MIN,9,18,400951603,1,10,-28
11185,3,[],(2:00 - 2nd) Two-Minute Warning,HOU,LAR,HOU,7,6,400951663,0,0,0


In [20]:
# Now look to extract the time remaining (in seconds)
detail = allplays_df.detail.values

# Make lists for quarter and time_remaining
qtr = []
time_rem = []

for d in detail[:]:
#    print(d)
#    print(type(d))
    try:
        if (not d) or (d == None):
            # detail is an empty list
            qtr.append(0)
            time_rem.append("0:00")
        
        else:
            pieces = d.split()
#            print(pieces)
            if pieces[0][0] == "E":
                # Found End of Quarter/Overtime line
                qtr.append(0)
                time_rem.append("0:00")

            elif pieces[0][0] == "(":
                # Found beginning of standard "(1:23 - 4th)" template
                qtr.append(pieces[2][0])
                time_rem.append(pieces[0].lstrip("("))
            
            else:
                # Not sure what this is, so just be safe and go to 0:00 rem in 4th
                print(d)
                qtr.append(0)
                time_rem.append("0:00")
            
    except:
        print("Default parse failed for:")
        print(d)
        qtr.append(0)
        time_rem.append("0:00")
            
#    print(qtr[-1])
#    print(time_rem[-1])
    
allplays_df['qtr'] = qtr
allplays_df['time_rem'] = time_rem

In [21]:
allplays_df[['downdist','detail','down','dist','qtr','time_rem']].sample(25)

,downdist,detail,down,dist,qtr,time_rem
96,['2nd and 10 at CAR 35'],(13:48 - 3rd) (Shotgun) C.Newton pass deep mi...,2,10,3,13:48
237,['3rd and 1 at MIA 8'],(6:15 - 2nd) (Shotgun) T.Coleman left guard t...,3,1,2,6:15
464,['3rd and 2 at GB 35'],(2:34 - 3rd) C.Keenum pass short right to K.R...,3,2,3,2:34
9473,['1st and Goal at DET 4'],"(10:24 - 4th) (No Huddle, Shotgun) T.Montgome...",1,Goal,4,10:24
4573,['1st and 10 at DEN 43'],(11:05 - 2nd) (Shotgun) P.Rivers pass short l...,1,10,2,11:05
8670,['3rd and 9 at ARI 47'],(6:03 - 1st) (Shotgun) C.Beathard sacked at S...,3,9,1,6:03
2539,['3rd and 3 at OAK 21'],(1:10 - 2nd) (Shotgun) A.Smith pass incomplet...,3,3,2,1:10
9382,[],END QUARTER 1,0,0,0,0:00
6479,['1st and 25 at CAR 48'],(6:14 - 2nd) (Shotgun) J.Winston pass short l...,1,25,2,6:14
4106,['1st and 10 at DAL 45'],(0:00 - 2nd) (Shotgun) E.Elliott up the middl...,1,10,2,0:00


In [22]:
# Make a column for secconds remaining in the game
qtr = allplays_df.qtr.values
time_rem = allplays_df.time_rem.values

secs_rem = []

for i, tr in enumerate(time_rem):
    if qtr[i] in ["1","2","3","4"]:
        q = int(qtr[i])
    elif qtr[i] == "O":
        q = 4
    else:
        q = 0
    mins = int(tr.split(":")[0])
    secs = int(tr.split(":")[1])
    secs_rem.append( 900*(4-q) + 60*mins + secs )
    
allplays_df['secs_rem'] = secs_rem

In [23]:
allplays_df[['qtr','time_rem','secs_rem']].sample(10)

,qtr,time_rem,secs_rem
2972,4,3:31,211
10851,3,3:29,1109
5357,4,6:43,403
8849,1,8:58,3238
5471,3,12:40,1660
6868,3,7:09,1329
6674,3,13:54,1734
2119,4,0:00,0
10456,2,5:15,2115
2579,3,3:38,1118


In [24]:
allplays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 15 columns):
Unnamed: 0       11935 non-null int64
downdist         11935 non-null object
detail           11935 non-null object
home             11935 non-null object
away             11935 non-null object
possession       11935 non-null object
home_score       11935 non-null int64
away_score       11935 non-null int64
gameId           11935 non-null int64
down             11935 non-null int64
dist             11935 non-null object
home_fieldpos    11935 non-null int64
qtr              11935 non-null object
time_rem         11935 non-null object
secs_rem         11935 non-null int64
dtypes: int64(7), object(8)
memory usage: 1.4+ MB


In [25]:
# Make column for score difference
allplays_df['home_lead'] = allplays_df['home_score'] - allplays_df['away_score']

# Make column for total score
allplays_df['total_score'] = allplays_df['home_score'] + allplays_df['away_score']

In [26]:
# Make column with derived metric for adjusted lead
# Make a column for adjusted score
import math
def adjusted_lead(play):
    try:
        return play.home_lead / math.sqrt( 3600-play.secs_rem + 1 )
    except:
        return 0
    
allplays_df['adj_lead'] = allplays_df.apply(
    lambda row: adjusted_lead(row), axis=1 )

In [27]:
# Make a column for whether this play takes place in overtime
allplays_df['OT'] = [1 if " OT" in str(p) else 0 for p in allplays_df.detail.values]

In [28]:
allplays_df[allplays_df.OT == 1].sample(10)

,Unnamed: 0,downdist,detail,home,away,possession,home_score,away_score,gameId,down,dist,home_fieldpos,qtr,time_rem,secs_rem,home_lead,total_score,adj_lead,OT
11093,3,['1st and 10 at JAX 41'],(8:46 - OT) (Shotgun) B.Bortles pass short ri...,LAC,JAX,JAX,17,17,400951648,1,10,9,O,8:46,526,0,34,0.0,1
3160,1,['2nd and 3 at CLE 42'],(5:24 - OT) D.Murray right guard to CLV 42 fo...,TEN,CLE,TEN,9,9,400951582,2,3,8,O,5:24,324,0,18,0.0,1
3146,4,['3rd and 10 at CLE 25'],(9:20 - OT) (Shotgun) C.Kessler pass incomple...,TEN,CLE,CLE,9,9,400951582,3,10,25,O,9:20,560,0,18,0.0,1
11097,7,['2nd and 8 at LAC 47'],(6:37 - OT) (Shotgun) B.Bortles pass incomple...,LAC,JAX,JAX,17,17,400951648,2,8,-3,O,6:37,397,0,34,0.0,1
3156,1,['2nd and 8 at CLE 22'],(6:54 - OT) C.Kessler pass incomplete short l...,TEN,CLE,CLE,9,9,400951582,2,8,28,O,6:54,414,0,18,0.0,1
3159,0,['1st and 10 at CLE 49'],(5:57 - OT) D.Murray left guard to CLV 42 for...,TEN,CLE,TEN,9,9,400951582,1,10,1,O,5:57,357,0,18,0.0,1
1277,3,['4th and 2 at BAL 48'],(5:48 - OT) (Punt formation) S.Koch punts 45 ...,CHI,BAL,BAL,24,24,400951699,4,2,2,O,5:48,348,0,48,0.0,1
11107,3,[],(3:17 - OT) Timeout #1 by LAC at 03:17.,LAC,JAX,JAX,17,17,400951648,0,0,0,O,3:17,197,0,34,0.0,1
11104,0,['1st and 10 at LAC 17'],(5:11 - OT) L.Fournette up the middle to LAC ...,LAC,JAX,JAX,17,17,400951648,1,10,-33,O,5:11,311,0,34,0.0,1
3155,0,['1st and 10 at CLE 20'],(7:30 - OT) I.Crowell right guard to CLV 22 f...,TEN,CLE,CLE,9,9,400951582,1,10,30,O,7:30,450,0,18,0.0,1


In [29]:
# Make column for whether home team has possession
hometeam = allplays_df.home.values
awayteam = allplays_df.away.values
possession = allplays_df.possession.values

home_possession = [
    1 if hometeam[i] == p else 0 if awayteam[i] == p else "X" for i, p in enumerate(possession)
]
allplays_df['home_possession'] = home_possession

In [30]:
# Make a column for whether the hoem team wins
gamedata_df.columns

Index(['result', 'season', 'week', 'home', 'away', 'winner', 'home_score',
       'away_score', 'OT'],
      dtype='object')

In [31]:
hometeam = gamedata_df.home.values
awayteam = gamedata_df.away.values
winner = gamedata_df.winner.values

# Make new column for gamedata
home_wins = [1 if hometeam[i] == w else 0 if awayteam[i] == w else "X" for i, w in enumerate(winner)]
gamedata_df['home_win'] = home_wins

# Try pandas join
#allplays_df = allplays_df.join(gamedata_df['home_win'], on='gameId')

In [32]:
# Must be an easier way to make the home_win column but hopefully this works
joined_df = pd.merge(allplays_df, gamedata_df[['season','week','home_win']], 
                     how='left',
                     left_on='gameId',
                     right_index=True)

In [33]:
joined_df.columns

Index(['Unnamed: 0', 'downdist', 'detail', 'home', 'away', 'possession',
       'home_score', 'away_score', 'gameId', 'down', 'dist', 'home_fieldpos',
       'qtr', 'time_rem', 'secs_rem', 'home_lead', 'total_score', 'adj_lead',
       'OT', 'home_possession', 'season', 'week', 'home_win'],
      dtype='object')

In [34]:
final_cols = [
    'downdist', 'detail', 'home', 'away', 'possession',
       'home_score', 'away_score', 'gameId', 'down', 'dist', 'home_fieldpos',
       'qtr', 'time_rem', 'secs_rem', 'home_lead', 'total_score', 'adj_lead',
       'OT','home_possession','home_win','season','week'
]
print(joined_df[final_cols].describe())
joined_df[final_cols].sample(20)

         home_score    away_score        gameId          down  home_fieldpos  \
count  11935.000000  11935.000000  1.193500e+04  11935.000000   11935.000000   
mean      10.125765     11.798911  4.009517e+08      1.748052      -0.314202   
std        9.667361     10.073924  7.194157e+01      1.163426      22.436372   
min        0.000000      0.000000  4.009516e+08      0.000000     -49.000000   
25%        2.000000      3.000000  4.009516e+08      1.000000     -17.000000   
50%        7.000000     10.000000  4.009517e+08      2.000000       0.000000   
75%       16.000000     17.000000  4.009518e+08      3.000000      16.000000   
max       51.000000     52.000000  4.009518e+08      4.000000      49.000000   

           secs_rem     home_lead   total_score      adj_lead            OT  \
count  11935.000000  11935.000000  11935.000000  11935.000000  11935.000000   
mean    1750.625052     -1.673146     21.924675     -0.069224      0.005279   
std     1071.011645     11.054847     16.3

,downdist,detail,home,away,possession,home_score,away_score,gameId,down,dist,...,time_rem,secs_rem,home_lead,total_score,adj_lead,OT,home_possession,home_win,season,week
427,['1st and 40 at MIN 40'],(0:48 - 2nd) (Shotgun) J.McKinnon up the midd...,GB,MIN,MIN,10,14,400951702,1,40,...,0:48,1848,-4,24,-0.095536,0,0,0,2017,6
8791,['2nd and 8 at SF 37'],(9:52 - 4th) A.Peterson up the middle to SF 3...,ARI,SF,ARI,17,10,400951763,2,8,...,9:52,592,7,27,0.127611,0,1,1,2017,9
9090,['1st and 10 at KC 18'],(3:20 - 2nd) (Shotgun) A.Smith pass deep midd...,KC,DAL,KC,3,7,400951786,1,10,...,3:20,2000,-4,10,-0.099969,0,1,0,2017,9
1555,['1st and Goal at ARI 6'],(9:18 - 3rd) D.Martin up the middle to ARZ 4 ...,TB,ARI,TB,0,31,400951770,1,Goal,...,9:18,1458,-31,31,-0.669654,0,1,0,2017,6
10535,['1st and 10 at IND 35'],(8:39 - 4th) (Run formation) J.Conner left en...,PIT,IND,PIT,17,17,400951565,1,10,...,8:39,519,0,34,0.000000,0,1,1,2017,10
10557,['1st and 10 at IND 18'],(0:35 - 4th) (Run formation) L.Bell right end...,PIT,IND,PIT,17,17,400951565,1,10,...,0:35,35,0,34,0.000000,0,1,1,2017,10
2641,['1st and Goal at KC 2'],(0:00 - 4th) (Shotgun) D.Carr pass short left...,KC,OAK,OAK,30,24,400951571,1,Goal,...,0:00,0,6,54,0.099986,0,0,0,2017,7
10447,['2nd and 10 at PIT 49'],(8:01 - 2nd) (Shotgun) J.Brissett pass short ...,PIT,IND,IND,0,7,400951565,2,10,...,8:01,2281,-7,7,-0.192669,0,0,1,2017,10
7638,['1st and Goal at TEN 5'],(1:01 - 4th) (Shotgun) J.Flacco pass incomple...,BAL,TEN,BAL,13,23,400951761,1,Goal,...,1:01,61,-10,36,-0.168073,0,1,0,2017,9
6467,['1st and 16 at CAR 7'],(10:18 - 2nd) (Shotgun) C.Newton up the middl...,CAR,TB,CAR,7,0,400951725,1,16,...,10:18,2418,7,7,0.203519,0,1,1,2017,8


In [35]:
joined_df[final_cols].to_csv("espn_parsedplays_2017_wk6-10.csv")

## End of finished code